# Download SEC filing data

In [7]:
from sec_edgar_downloader import Downloader
import os, os.path
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Initialize a downloader instance. If no argument is passed
# to the constructor, the package will download filings to
# the current working directory.

currentDirectory = os.getcwd()
print(currentDirectory)
dl = Downloader(currentDirectory + '/data/download_2/')

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung


In [55]:
DIR = currentDirectory + '/data/download_2/sec-edgar-filings/'
for file in os.listdir(DIR):
    if not file.startswith('.'):
        company_dir = os.path.join(DIR, file,'10-K/')
        print('10K - ' + file + ': ' + str(len(os.listdir(company_dir))))
for file in os.listdir(DIR):
    if not file.startswith('.'):
        company_dir = os.path.join(DIR, file,'10-Q/')
        print('10Q - ' + file + ': ' + str(len(os.listdir(company_dir))))
    
#0001751788
#0001744489
#10Q - 0001618921
#10Q - 0000104169:
#10Q - 0001751788
#10Q - 0001744489:

10K - 0000354950: 6
10K - 0000018230: 6
10K - 0000051143: 7
10K - 0001618921: 7
10K - 0000086312: 6
10K - 0000004962: 6
10K - 0000773840: 6
10K - 0001108524: 6
10K - 0000012927: 6
10K - 0000104169: 6
10K - 0001403161: 7
10K - 0001751788: 3
10K - 0000732712: 7
10K - 0000731766: 6
10K - 0000021344: 6
10K - 0000063908: 7
10K - 0000080424: 6
10K - 0000320193: 6
10K - 0000886982: 6
10K - 0000318154: 6
10K - 0000050863: 6
10K - 0000310158: 6
10K - 0000066740: 6
10K - 0000858877: 6
10K - 0000200406: 6
10K - 0000320187: 6
10K - 0000019617: 6
10K - 0000093410: 7
10K - 0000789019: 7
10K - 0001744489: 3
10Q - 0000354950: 18
10Q - 0000018230: 18
10Q - 0000051143: 18
10Q - 0001618921: 19
10Q - 0000086312: 19
10Q - 0000004962: 19
10Q - 0000773840: 18
10Q - 0001108524: 18
10Q - 0000012927: 18
10Q - 0000104169: 18
10Q - 0001403161: 18
10Q - 0001751788: 9
10Q - 0000732712: 18
10Q - 0000731766: 18
10Q - 0000021344: 18
10Q - 0000063908: 18
10Q - 0000080424: 18
10Q - 0000320193: 18
10Q - 0000886982: 18
10

# Download dow30 10-K 10-Q

In [3]:
filename = currentDirectory + '/dow30_info.csv'
dow30_df = pd.read_csv(filename,sep=",")

In [4]:
#for index, row in dow30_df.iterrows():
#    print(row['Symbol'], row['CIK'])
#    print(str(row['CIK']).zfill(10))

In [45]:
CIK_list = dow30_df['CIK'].tolist()

In [53]:
CIK_list = [1751788,1744489]

In [54]:
for x in CIK_list: 
    dl.get("10-Q",x,amount=18)
    dl.get("10-K",x,amount=6)

# Functions to parse report

In [57]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 3 if not w in stopwords.words('english')] ## change to 3 
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)



In [58]:
def parse_10k(company, filepath): 
    f = open(filepath)
    raw_10k = f.read()
    
    ## find start end range to avoid xml parsing problems ##
    import re
    # Regex to find <DOCUMENT> tags
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
    
    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
    
    document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-K':
            #print(doc_start)
            #print(doc_end)
            doc_10k = raw_10k[doc_start:doc_end].lower()
            break
            
    doc_10k_bs = BeautifulSoup(doc_10k, 'lxml')
    doc_10k_str = str(doc_10k_bs)
    
    #i7 = "management’s discussion and analysis of financial condition and results of operations"
    i7 = "s discussion and analysis of financial condition and results of operations"
    i7a = 'quantitative and qualitative disclosures about market risk' 
    #if company == 'GS':
    #    i7 = 's discussion and analysis of financial condition and results of operations'
    #    i7a = 'quantitative and qualitative disclosure about market risk' 
    if company == 'INTC':
        i7 = 's discussion and analysis \(md&amp;a\)'
        i7a = 'font-weight:bold;">other key information'
    if company == 'JNJ':
        i7 = "s discussion and analysis of results of operations and financial condition"
    if company == 'MSFT':
        if read_file_year(filepath) == '2016':
            i7 = 's discussion and analysis of financial condition and results of operations </b>'
            i7a = 'market risk </b>'
        else: 
            i7a = 'ive disclosures about market risk'
    if company == 'IBM':
        i7 = 'item\xa07. management’s discussion and analysis of financial condition and results of operations.<'
    if company == 'MCD':
        if int(read_file_year(filepath)) >= 2020:
            i7 = "s discussion and analysis of financial condition and results of operations</"
            i7a = 'other key information'
        else:
            i7 = 'item\xa07. management’s discussion and analysis of financial condition and results of operations'
    if company == 'CVX':
        i7 = 'd;">key financial results'
        i7a = 'd;">quarterly results'
    if company == 'JPM':
        if read_file_year(filepath) == '2017':
            i7 = 'introduction </font>'
            i7a = 's report on internal control over financial reporting</'
        else:
            i7 = 'introduction </span></'
            i7a = 's report on internal control over financial reporting</span></div>'
    if company == 'V':
        if int(read_file_year(filepath)) >= 2019:
            i7 = 's discussion and analysis of financial condition and results of operations</span></div>'
            i7a = 'quantitative and qualitative disclosures about market risk</span></div>'
        else: 
            i7 = 'd;">management’s discussion and analysis of financial condition and results of operations</font></div></td>'
            i7a = 'quantitative and qualitative disclosures about market risk</font></div>'
    if company == 'WBA':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosure about market risk</'

        
    
    sec7_find = re.finditer(i7, doc_10k_str)
    sec7a_find = re.finditer(i7a, doc_10k_str)
    
    # moving keyword range to dataframe
    sec7_start_range = []
    sec7_end_range = []
    for x in sec7_find: 
        print('7: ' + str(x.start()))
        sec7_start_range.append(x.start())
        sec7_end_range.append(x.end())
        sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
    if not sec7_start_range: 
        print('no item 7')
        return sec7_start_range
    sec7a_start_range = []
    sec7a_end_range = []    
    for x in sec7a_find: 
        print('7a: ' + str(x.start()))
        sec7a_start_range.append(x.start())
        sec7a_end_range.append(x.end())
        sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})
    if not sec7a_start_range: 
        print('no item 7a')
        return sec7a_start_range
    
    #qc check
    #for index, row in sec7_df.iterrows():
    #    print('7' , str(row['start']) , str(row['end']))
    #for index, row in sec7a_df.iterrows():
    #    print('7a', str(row['start']), str(row['end']))
        
    # get possible ranges 
    possible_start_range = []
    possible_end_range = []
    for index, row7a in sec7a_df.iterrows():
        #print(row7a['start'])
        filtered = []
        for i, row7 in sec7_df.iterrows():
            #print(row7['end'])
            if row7['end'] < row7a['start']:
                filtered.append(row7['end'])
        #print(filtered)
        if not filtered:
            continue # nothing before item 7a
        max_y = max(filtered)
        possible_start_range.append(max_y)
        possible_end_range.append(row7a['start']-12)
    
        # qc 
        #for x in possible_start_range:
        #    print(x)

    # get the largest possible range
    possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
    possible_df['delta'] = possible_df['end'] - possible_df['start']
    if len(possible_df) == 0:
        print('exee')
        return sec7a_start_range
    print(possible_df.loc[possible_df['delta'].idxmax()])

    item7_df = possible_df.loc[possible_df['delta'].idxmax()]
    
    item_7_content = doc_10k_str[item7_df['start']:item7_df['end']]
    item_7_cleaned = preprocess(item_7_content)
    bagOfWordsA = item_7_cleaned.split(' ')
    
    return bagOfWordsA


In [59]:
def parse_10q(company, filepath): 
    f = open(filepath)
    raw_10k = f.read()
    
    ## find start end range to avoid xml parsing problems ##
    import re
    # Regex to find <DOCUMENT> tags
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
    
    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
    
    document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-Q':
            #print(doc_start)
            #print(doc_end)
            doc_10k = raw_10k[doc_start:doc_end].lower()
            break
            
    doc_10k_bs = BeautifulSoup(doc_10k, 'lxml')
    doc_10k_str = str(doc_10k_bs)
    
    #i7 = "management’s discussion and analysis of financial condition and results of operations"
    i7 = "s discussion and analysis of financial condition and results of operations"
    i7a = 'quantitative and qualitative disclosures about market risk' 
    if company == 'INTC':
        i7 = 's discussion and analysis'
        i7a = 'font-weight:bold;">other key information'
    if company == 'IBM':
        i7 = 's discussion and analysis of results of operations and financial condition'
        i7a = 'controls and procedures'
    if company == 'AMGN':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'DOW':
        i7 = 's discussion and analysis of financial condition and results of operations.</span>'
        i7a = 'quantitative and qualitative disclosures about market risk</span>'
    if company == 'MMM':
        i7 ='s discussion and analysis of financial condition and results of operations.</'
    if company == 'MRK':
        i7 = 's discussion and analysis of financial condition and results of operations</span>'
        i7a = 'item\xa04. controls and procedures'
    if company == 'NKE':
        i7 = 's discussion and analysis of financial condition and results of operations</span>'
        i7a = 'quantitative and qualitative disclosures about market risk</span>'
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'UNH':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'TRV':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'MSFT':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosures about market risk</'
    if company == 'HON' and read_file_year(filepath) == '2020': # format changed after 2021
        i7 = 's discussion and analysis of<'
    if company == 'WBA':
        i7 = 's discussion and analysis of financial condition and results of operations</'
        i7a = 'quantitative and qualitative disclosure about market risk</'
        #i7a = 'quantitative and qualitative disclosure about market risk</'
        
    sec7_find = re.finditer(i7, doc_10k_str)
    sec7a_find = re.finditer(i7a, doc_10k_str)
    
    
    # moving keyword range to dataframe
    sec7_start_range = []
    sec7_end_range = []
    for x in sec7_find: 
        print('7: ' + str(x.start()))
        sec7_start_range.append(x.start())
        sec7_end_range.append(x.end())
        sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
    if not sec7_start_range: 
        print('no item 2')
        return sec7_start_range
    sec7a_start_range = []
    sec7a_end_range = []    
    for x in sec7a_find: 
        print('7a: ' + str(x.start()))
        sec7a_start_range.append(x.start())
        sec7a_end_range.append(x.end())
        sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})
    if not sec7a_start_range:
        print('no item 3')
        return sec7a_start_range
    
    #qc check
    #for index, row in sec7_df.iterrows():
    #    print('7' , str(row['start']) , str(row['end']))
    #for index, row in sec7a_df.iterrows():
    #    print('7a', str(row['start']), str(row['end']))
        
    # get possible ranges 
    possible_start_range = []
    possible_end_range = []
    for index, row7a in sec7a_df.iterrows():
        #print(row7a['start'])
        filtered = []
        for i, row7 in sec7_df.iterrows():
            #print(row7['end'])
            if row7['end'] < row7a['start']:
                filtered.append(row7['end'])
        #print(filtered)
        if not filtered:
            continue # nothing before item 7a
        max_y = max(filtered)
        possible_start_range.append(max_y)
        possible_end_range.append(row7a['start']-12)
    
        # qc 
        #for x in possible_start_range:
        #    print(x)

    # get the largest possible range
    possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
    possible_df['delta'] = possible_df['end'] - possible_df['start']
    print(possible_df.loc[possible_df['delta'].idxmax()])

    item7_df = possible_df.loc[possible_df['delta'].idxmax()]
    
    item_7_content = doc_10k_str[item7_df['start']:item7_df['end']]
    item_7_cleaned = preprocess(item_7_content)
    bagOfWordsA = item_7_cleaned.split(' ')
    
    return bagOfWordsA


In [60]:
def read_file_date(filepath):
    # Regex to find <DOCUMENT> tags
    date_start_pattern = re.compile(r'FILED AS OF DATE:')
    date_end_pattern = re.compile(r'DATE AS OF CHANGE:')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    date_type_pattern = re.compile(r'<TYPE>[^\n]+')

    f = open(filepath, "r")

    raw_10k = f.read()
    document = {}

    date_start_is = [x.end() for x in date_start_pattern.finditer(raw_10k)]
    date_end_is = [x.start() for x in date_end_pattern.finditer(raw_10k)]
    date_types = [x[len('<TYPE>'):] for x in date_type_pattern.findall(raw_10k)]

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for date_type, date_start, date_end in zip(date_types, date_start_is, date_end_is):
        if date_type == '10-K':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
        if date_type == '10-Q':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
    return ddd

In [61]:
def read_file_year(filepath):
    # Regex to find <DOCUMENT> tags
    date_start_pattern = re.compile(r'FILED AS OF DATE:')
    date_end_pattern = re.compile(r'DATE AS OF CHANGE:')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    date_type_pattern = re.compile(r'<TYPE>[^\n]+')

    f = open(filepath, "r")

    raw_10k = f.read()
    document = {}

    date_start_is = [x.end() for x in date_start_pattern.finditer(raw_10k)]
    date_end_is = [x.start() for x in date_end_pattern.finditer(raw_10k)]
    date_types = [x[len('<TYPE>'):] for x in date_type_pattern.findall(raw_10k)]

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for date_type, date_start, date_end in zip(date_types, date_start_is, date_end_is):
        if date_type == '10-K':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
            yyy = ddd[0:4]
        if date_type == '10-Q':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
            yyy = ddd[0:4]
    return yyy

In [62]:
def read_conformed_period_date(filepath):
    # Regex to find <DOCUMENT> tags
    date_start_pattern = re.compile(r'CONFORMED PERIOD OF REPORT:')
    date_end_pattern = re.compile(r'FILED AS OF DATE:')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    date_type_pattern = re.compile(r'<TYPE>[^\n]+')

    f = open(filepath, "r")

    raw_10k = f.read()
    document = {}

    date_start_is = [x.end() for x in date_start_pattern.finditer(raw_10k)]
    date_end_is = [x.start() for x in date_end_pattern.finditer(raw_10k)]
    date_types = [x[len('<TYPE>'):] for x in date_type_pattern.findall(raw_10k)]

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for date_type, date_start, date_end in zip(date_types, date_start_is, date_end_is):
        if date_type == '10-K':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
        if date_type == '10-Q':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
    return ddd

In [63]:
read_file_year('/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-K/0001403161-19-000050/full-submission.txt')

'2019'

## 10-K read using CIK

In [64]:
companyk = []
file_namek = []
doc_typek = []
file_datek = []
contentk = []
cikk = []
conformed_periodk = []
bag_lenk = []
#for x in dow30_df.: # company loop
sym_list = ['INTC', 'IBM', 'VZ', 'HON']
dow30_df2 = dow30_df[dow30_df['Symbol'].isin(sym_list)]
for index, row in dow30_df2.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-K/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=5: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10k(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenk.append(len(item7))
            contentk.append(item7)
            cikk.append(x)
            companyk.append(row['Symbol'])
            file_namek.append(name)
            doc_typek.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_datek.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodk.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000773840/10-K/0000773840-20-000009/full-submission.txt
HON
7: 588293
7: 661091
7: 745374
7: 800483
7: 800965
7: 1258731
7a: 590180
7a: 1258442
start     801039
end      1258430
delta     457391
Name: 1, dtype: int64
4368
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000773840/10-K/0000930413-18-000292/full-submission.txt
HON
7: 19968
7: 64133
7: 141732
7: 142125
7: 448770
7a: 20635
7a: 448521
start    142199
end      448509
delta    306310
Name: 1, dtype: int64
5375
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000773840/10-K/0000930413-19-000366/full-submission.txt
HON
7: 19608
7: 62804
7: 142216
7: 142609
7: 450495
7a: 20275
7a: 450242
start    142683
end      450230
delta    307547
Name: 1, dtype: int64
5106
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/down

In [19]:
# CVX, INTC, IBM, MSFT, .5 VZ
# CVX, HON download 1 more
# IBM no item 7, they put it in annual report 
# VZ no item 7 before 2021, they put it in annual report 



## 10-Q read using CIK

In [65]:
companyq = []
file_nameq = []
doc_typeq = []
file_dateq = []
contentq = []
cikq = []
conformed_periodq = []
bag_lenq = []
#for x in dow30_df.: # company loop
for index, row in dow30_df.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-Q/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=10: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10q(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenq.append(len(item7))
            contentq.append(item7)
            cikq.append(x)
            companyq.append(row['Symbol'])
            file_nameq.append(name)
            doc_typeq.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_dateq.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodq.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-16-007105/full-submission.txt
MMM
7: 3729589
7a: 50415
7a: 5083902
7a: 5084684
start    3729666
end      5084672
delta    1355006
Name: 1, dtype: int64
6828
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-18-005773/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-17-007801/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-18-004248/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000066740/10-Q/0001558370-16-008937/full-submission.txt
MMM
no item 2
0
/Users/wailunchung/Documents/GitHu

7: 216461
7: 1479528
7a: 217882
7a: 1655163
7a: 1655632
start    1479602
end      1655620
delta     176018
Name: 2, dtype: int64
2517
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320193/10-Q/0000320193-17-000009/full-submission.txt
AAPL
7: 26297
7: 1258756
7: 1701345
7a: 27237
7a: 1691734
7a: 1692483
start    1258830
end      1692471
delta     433641
Name: 2, dtype: int64
3861
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320193/10-Q/0000320193-18-000100/full-submission.txt
AAPL
7: 27481
7: 1144041
7: 1560455
7a: 28561
7a: 1550556
7a: 1551305
start    1144115
end      1551293
delta     407178
Name: 2, dtype: int64
4174
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320193/10-Q/0000320193-20-000010/full-submission.txt
AAPL
7: 205015
7: 1373485
7a: 206095
7a: 1538367
7a: 1539117
start    1373559
end      1539105
delta     

7: 22462
7: 3362357
7: 4145425
7a: 23585
7a: 5121155
start    4145499
end      5121143
delta     975644
Name: 1, dtype: int64
3480
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000093410/10-Q/0000093410-17-000019/full-submission.txt
CVX
7: 28215
7: 785233
7a: 29124
7a: 1053342
start     785307
end      1053330
delta     268023
Name: 1, dtype: int64
3132
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000093410/10-Q/0000093410-16-000083/full-submission.txt
CVX
7: 25676
7: 953507
7a: 26585
7a: 1383920
start     953581
end      1383908
delta     430327
Name: 1, dtype: int64
3790
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000093410/10-Q/0000093410-16-000070/full-submission.txt
CVX
7: 25389
7: 947047
7a: 26298
7a: 1381362
start     947121
end      1381350
delta     434229
Name: 1, dtype: int64
3679
/Users/wailunchung/Documents/GitHub

6590
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000021344/10-Q/0000021344-20-000046/full-submission.txt
KO
7: 3053681
7a: 380155
7a: 3380215
start    3053755
end      3380203
delta     326448
Name: 0, dtype: int64
7948
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000021344/10-Q/0000021344-17-000026/full-submission.txt
KO
7: 1487432
7a: 33167
7a: 1973075
start    1487506
end      1973063
delta     485557
Name: 0, dtype: int64
10303
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000021344/10-Q/0000021344-20-000041/full-submission.txt
KO
7: 3022244
7a: 328360
7a: 3370112
start    3022318
end      3370100
delta     347782
Name: 0, dtype: int64
7549
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000021344/10-Q/0000021344-17-000019/full-submission.txt
KO
7: 1020850
7a: 32851
7a: 13

7: 104161
7: 120753
7: 130416
7: 750193
7a: 105378
7a: 923342
start    750267
end      923330
delta    173063
Name: 1, dtype: int64
2323
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000354950/10-Q/0000354950-20-000068/full-submission.txt
HD
7: 126858
7: 149010
7: 159524
7: 1053846
7a: 128428
7a: 1300846
start    1053920
end      1300834
delta     246914
Name: 1, dtype: int64
2459
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000354950/10-Q/0000354950-21-000189/full-submission.txt
HD
7: 99127
7: 115719
7: 125351
7: 724997
7a: 100344
7a: 897607
start    725071
end      897595
delta    172524
Name: 1, dtype: int64
2301
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000354950/10-Q/0000354950-18-000051/full-submission.txt
HD
7: 33905
7: 51776
7: 62508
7: 411042
7a: 34808
7a: 641444
start    411116
end      641432
delta    230316
Name:

...

6227
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000063908/10-Q/0000063908-18-000064/full-submission.txt
MCD
7: 114996
7: 1008900
7: 1742272
7a: 116185
7a: 1737046
start    1008974
end      1737034
delta     728060
Name: 1, dtype: int64
6441
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000063908/10-Q/0000063908-16-000121/full-submission.txt
MCD
7: 30979
7: 350420
7: 749263
7a: 32288
7a: 744847
start    350494
end      744835
delta    394341
Name: 1, dtype: int64
5542
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000063908/10-Q/0000063908-20-000063/full-submission.txt
MCD
7: 140852
7: 1199490
7: 1916792
7a: 142041
7a: 1911279
start    1199564
end      1911267
delta     711703
Name: 1, dtype: int64
6905
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000063908/10-Q/0000063908-20

6271
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320187/10-Q/0000320187-19-000007/full-submission.txt
NKE
7: 30103
7: 1627302
7a: 31024
7a: 2451265
start    1627378
end      2451253
delta     823875
Name: 1, dtype: int64
5990
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320187/10-Q/0000320187-16-000264/full-submission.txt
NKE
7: 27723
7: 1189315
7a: 28644
7a: 2216452
start    1189391
end      2216440
delta    1027049
Name: 1, dtype: int64
7927
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320187/10-Q/0000320187-17-000031/full-submission.txt
NKE
7: 28456
7: 1188343
7a: 29377
7a: 2208097
start    1188419
end      2208085
delta    1019666
Name: 1, dtype: int64
6949
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000320187/10-Q/0000320187-20-000004/full-submission.txt
NKE

7: 305138
7: 3141177
7a: 307773
7a: 4228334
start    3141253
end      4228322
delta    1087069
Name: 1, dtype: int64
11594
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-Q/0000086312-18-000008/full-submission.txt
TRV
7: 31587
7: 2130250
7a: 33138
7a: 2941258
start    2130326
end      2941246
delta     810920
Name: 1, dtype: int64
8745
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-Q/0001104659-16-113040/full-submission.txt
TRV
no item 2
0
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-Q/0000086312-19-000035/full-submission.txt
TRV
7: 352476
7: 4073693
7a: 354027
7a: 5256592
start    4073769
end      5256580
delta    1182811
Name: 1, dtype: int64
10387
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000086312/10-Q/0000086312-18-000038/full-submi

7: 24896
7: 878716
7: 1104900
7a: 25745
7a: 1150677
start    1104974
end      1150665
delta      45691
Name: 1, dtype: int64
516
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-Q/0001403161-22-000013/full-submission.txt
V
7: 169562
7: 1393700
7a: 170700
7a: 1691518
start    1393774
end      1691506
delta     297732
Name: 1, dtype: int64
2344
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-Q/0001403161-19-000017/full-submission.txt
V
7: 24909
7: 1395120
7: 1810310
7a: 25758
7a: 1857613
start    1810384
end      1857601
delta      47217
Name: 1, dtype: int64
576
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001403161/10-Q/0001403161-20-000012/full-submission.txt
V
7: 158552
7: 1241619
7a: 159401
7a: 1535462
start    1241693
end      1535450
delta     293757
Name: 1, dtype: int64
2006
/Users/wailunchung/Documen

7: 4420180
7: 5614675
7a: 5614393
start    4420254
end      5614381
delta    1194127
Name: 0, dtype: int64
7468
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001744489/10-Q/0001744489-20-000046/full-submission.txt
DIS
7: 3684539
7: 4486847
7a: 4486382
start    3684613
end      4486370
delta     801757
Name: 0, dtype: int64
5146
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001744489/10-Q/0001744489-20-000151/full-submission.txt
DIS
7: 4086015
7: 5617880
7a: 5617576
start    4086089
end      5617564
delta    1531475
Name: 0, dtype: int64
10534
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001744489/10-Q/0001744489-20-000102/full-submission.txt
DIS
7: 4107935
7: 5648529
7a: 5648064
start    4108009
end      5648052
delta    1540043
Name: 0, dtype: int64
9951
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/downl

### combining 10k and 10q

In [66]:
company = []
file_name = []
doc_type = []
file_date = []
content = []
cik = []
conformed_period = []
bag_len = []
company = companyk + companyq
file_name = file_namek+ file_nameq
doc_type = doc_typek + doc_typeq
file_date = file_datek + file_dateq
content = contentk + contentq
cik = cikk + cikq
conformed_period = conformed_periodk + conformed_periodq
bag_len = bag_lenk + bag_lenq

In [67]:
all_df = pd.DataFrame(
    {'company': company,
     'cik': cik,
     'file_name': file_name,
     'doc_type': doc_type,
     'file_date': file_date,
     'conformed_period': conformed_period,
     'bag_len': bag_len,
     'content': content
    })

## check how many pass minimum word count 


In [68]:
all_df

,company,cik,file_name,doc_type,file_date,conformed_period,bag_len,content
0,HON,773840,0000773840-20-000009,10-K,20200214,20191231,4368,"[intended, help, reader, understand, results, ..."
1,HON,773840,0000930413-18-000292,10-K,20180209,20171231,5375,"[intended, help, reader, understand, results, ..."
2,HON,773840,0000930413-19-000366,10-K,20190208,20181231,5106,"[intended, help, reader, understand, results, ..."
3,HON,773840,0000773840-22-000018,10-K,20220211,20211231,4569,"[income, attributable, honeywell, compared, ea..."
4,HON,773840,0000930413-17-000391,10-K,20170210,20161231,1,[21271]
...,...,...,...,...,...,...,...,...
313,DIS,1744489,0001744489-21-000108,10-K,20210513,20210403,9165,"[organization, informationmanagement, discussi..."
314,DIS,1744489,0001744489-19-000167,10-K,20190806,20190629,7905,"[organization, informationmanagement, discussi..."
315,DIS,1744489,0001744489-21-000047,10-K,20210211,20210102,6200,"[organization, informationmanagement, discussi..."
316,DIS,1744489,0001744489-22-000059,10-K,20220209,20220101,7051,"[quarterly, report, form, contains, forward, l..."


In [69]:
all_df[all_df.bag_len > 500]

,company,cik,file_name,doc_type,file_date,conformed_period,bag_len,content
0,HON,773840,0000773840-20-000009,10-K,20200214,20191231,4368,"[intended, help, reader, understand, results, ..."
1,HON,773840,0000930413-18-000292,10-K,20180209,20171231,5375,"[intended, help, reader, understand, results, ..."
2,HON,773840,0000930413-19-000366,10-K,20190208,20181231,5106,"[intended, help, reader, understand, results, ..."
3,HON,773840,0000773840-22-000018,10-K,20220211,20211231,4569,"[income, attributable, honeywell, compared, ea..."
5,INTC,50863,0000050863-19-000007,10-K,20190201,20181229,5427,"[five, years, strategy, transform, centric, da..."
...,...,...,...,...,...,...,...,...
313,DIS,1744489,0001744489-21-000108,10-K,20210513,20210403,9165,"[organization, informationmanagement, discussi..."
314,DIS,1744489,0001744489-19-000167,10-K,20190806,20190629,7905,"[organization, informationmanagement, discussi..."
315,DIS,1744489,0001744489-21-000047,10-K,20210211,20210102,6200,"[organization, informationmanagement, discussi..."
316,DIS,1744489,0001744489-22-000059,10-K,20220209,20220101,7051,"[quarterly, report, form, contains, forward, l..."


In [70]:
df_500 = all_df[all_df.bag_len > 500]

## save to CSV

In [75]:
df_500.to_csv('data_10k10q.csv', sep='|',index=False)

In [76]:
## another way ##

In [23]:
import csv

with open('data_10k10q.csv', 'w') as f:
    writer = csv.writer(f,delimiter ='|') # pipe delimited
    writer.writerow(['company', 'file_name', 'doc_type','file_date', 'cik', 'conformed_period','bag_len','content'])
    writer.writerows(zip(company, file_name, doc_type, file_date, cik, conformed_period, bag_len, content))

In [74]:
################ BELOW are parsing test ##########################

## TEST 1 company 10-K

In [472]:
companyq = []
file_nameq = []
doc_typeq = []
file_dateq = []
contentq = []
cikq = []
conformed_periodq = []
bag_lenq = []
#dow30_df_test = 0001618921
dow30_df_test = dow30_df.loc[dow30_df['CIK'] == 1618921]
#for x in dow30_df.: # company loop
for index, row in dow30_df_test.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-K/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=10: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10k(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenq.append(len(item7))
            contentq.append(item7)
            cikq.append(x)
            companyq.append(row['Symbol'])
            file_nameq.append(name)
            doc_typeq.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_dateq.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodq.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-21-000085/full-submission.txt
WBA
7: 387115
7: 723526
7a: 388293
7a: 1328222
start     723602
end      1328210
delta     604608
Name: 1, dtype: int64
9303
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001628280-18-012472/full-submission.txt
WBA
7: 32800
7: 526262
7a: 33728
7a: 1228684
start     526338
end      1228672
delta     702334
Name: 1, dtype: int64
8180
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-20-000082/full-submission.txt
WBA
7: 378561
7: 871567
7a: 380169
7a: 1657055
start     871643
end      1657043
delta     785400
Name: 1, dtype: int64
9683
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-19-000069/full-submission.txt
WBA
7: 

## TEST 1 company 10-Q

In [159]:
companyq = []
file_nameq = []
doc_typeq = []
file_dateq = []
contentq = []
cikq = []
conformed_periodq = []
bag_lenq = []
#dow30_df_test = 0001618921
dow30_df_test = dow30_df.loc[dow30_df['CIK'] == 789019]
#for x in dow30_df.: # company loop
for index, row in dow30_df_test.iterrows():
    x = row['CIK']
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download_2/'
    file_dir = file_dir + 'sec-edgar-filings/' + str(x).zfill(10) + '/10-Q/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=10: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            print(row['Symbol'])
            item7 = parse_10q(row['Symbol'], os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            bag_lenq.append(len(item7))
            contentq.append(item7)
            cikq.append(x)
            companyq.append(row['Symbol'])
            file_nameq.append(name)
            doc_typeq.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_dateq.append(file_d)
            conformed_period_d = read_conformed_period_date(os.path.join(file_dir,name)+'/full-submission.txt')
            conformed_periodq.append(conformed_period_d)
            i = i+1
            #print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-21-051992/full-submission.txt
MSFT
7: 245100
7: 3241404
7a: 247225
7a: 3659649
start    3241480
end      3659637
delta     418157
Name: 1, dtype: int64
4669
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-19-001392/full-submission.txt
MSFT


Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x127b5b400>
Traceback (most recent call last):
  File "/Users/wailunchung/.pyenv/versions/3.7.3/lib/python3.7/site-packages/nltk/data.py", line 1161, in __del__
    if not self.closed:
  File "/Users/wailunchung/.pyenv/versions/3.7.3/lib/python3.7/site-packages/nltk/data.py", line 1175, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


7: 44310
7a: 46363
start    44386
end      46351
delta     1965
Name: 0, dtype: int64
6
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-20-002450/full-submission.txt
MSFT
7: 297508
7: 4130943
7a: 299633
7a: 4778313
start    4131019
end      4778301
delta     647282
Name: 1, dtype: int64
4536
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-19-012709/full-submission.txt
MSFT
7: 44126
7a: 46179
start    44202
end      46167
delta     1965
Name: 0, dtype: int64
6
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-21-020891/full-submission.txt
MSFT
7: 327232
7: 4301797
7a: 329357
7a: 4973811
start    4301873
end      4973799
delta     671926
Name: 1, dtype: int64
5762
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/00

# step by step testing

In [7]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

3000


In [431]:
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/CAT/10-K/0000018230-21-000063/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/GS/10-K/0001193125-21-102511/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/IBM/10-K/0001558370-20-001786/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/MRK/10-Q/0000310158-20-000009/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/HON/10-Q/0000773840-20-000038/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000789019/10-Q/0001564590-19-012709/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000773840/10-K/0000930413-17-000391/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000019617/10-K/0000019617-20-000257/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0000019617/10-K/0000019617-21-000236/full-submission.txt'
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download_2/sec-edgar-filings/0001618921/10-K/0001618921-17-000069/full-submission.txt'


In [432]:

f = open(filepath)
raw_10k = f.read()

In [433]:
import re
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

In [434]:
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

In [435]:
document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        print(doc_start)
        print(doc_end)
        #doc_10k = raw_10k[doc_start:doc_end].lower()
        document[doc_type] = raw_10k[doc_start:doc_end].lower()
        doc_10k = raw_10k[doc_start:doc_end].lower()
        break

986
3343971


In [436]:
len(document['10-K'])

3342985

In [437]:
doc_10k = BeautifulSoup(doc_10k, 'lxml')

In [438]:
doc_10k_str = str(doc_10k)

In [465]:

i7 = 's discussion and analysis of financial condition and results of operations'
#i7 = 's discussion and analysis of financial condition and results of operations'
#JPM
#i7 = 'introduction'
#i7 = 'introduction </span></div>'
#MSFT 2016 
#i7 = 's discussion and analysis of financial condition and results of operations </b>'
# V 2019+
#i7 = 's discussion and analysis of financial condition and results of operations</span></div>'
#i7a = 'quantitative and qualitative disclosures about market risk</span></div>'
# V 2019<
#i7 = 'd;">management’s discussion and analysis of financial condition and results of operations</font></div></td>'
#i7a = 'quantitative and qualitative disclosures about market risk</font></div>'
# WBA
i7 = 's discussion and analysis of financial condition and results of operations</'
sec7_find = re.finditer(i7, doc_10k_str)
i7a = 'quantitative and qualitative disclosures about market risk'
#i7a = 'quantitative and qualitative disclosure about market risk'
i7a = 'quantitative and qualitative disclosures about market risk'
# WBA
i7 = 's discussion and analysis of financial condition and results of operations</'
i7a = 'quantitative and qualitative disclosure about market risk</'


## JPM 
#i7a = 's report on internal control over financial reporting'
#i7a = 's report on internal control over financial reporting</span></div>'
# MCD 
#i7a = 'other key information'
#i7a = 'item\xa04. controls and procedures'

sec7a_find = re.finditer(i7a, doc_10k_str)

#for x in sec7a_find: 
#    print(x)
sec7_df = None
sec7a_df = None
if sec7_df is not None:
    del sec7_df
if sec7a_df is not None:
    del sec7a_df

    # moving keyword range to dataframe
sec7_start_range = []
sec7_end_range = []
for x in sec7_find: 
    #print(x.start())
    sec7_start_range.append(x.start())
    sec7_end_range.append(x.end())
    sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
sec7a_start_range = []
sec7a_end_range = []    
for x in sec7a_find: 
    sec7a_start_range.append(x.start())
    sec7a_end_range.append(x.end())
    sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})


In [466]:
#del sec7a_df

In [467]:
for index, row in sec7_df.iterrows():
    print('7' , str(row['start']) , str(row['end']))
for index, row in sec7a_df.iterrows():
    print('7a', str(row['start']), str(row['end']))


7 32944 33020
7 518561 518637
7a 1209636 1209695


In [468]:
for index, row in sec7_df.iterrows(): 
    print(doc_10k_str[row['start']-100:row['end']+100])
    print('------')
print('####')
for index, row in sec7a_df.iterrows(): 
    print(doc_10k_str[row['start']-100:row['end']+100])
    print('------')


="font-family:inherit;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;">management’s discussion and analysis of financial condition and results of operations</font></a></div></td><td style="vertical-align:top;background-color:#cceeff;padding-left:2px;padding-
------
t style="font-family:inherit;font-size:10pt;font-weight:bold;text-decoration:underline;">management’s discussion and analysis of financial condition and results of operations</font></div><div style="line-height:120%;text-align:left;font-size:10pt;"><font style="font-family:in
------
####
</font><font style="font-family:inherit;font-size:10pt;font-weight:bold;text-decoration:underline;">quantitative and qualitative disclosure about market risk</font></div><div style="line-height:120%;text-align:left;font-size:10pt;"><font style="font-family:in
------


In [242]:
item_7_content = doc_10k_str[507977:548190]
sss = preprocess(item_7_content)
bagOfWordsA = sss.split(' ')

In [243]:
len(bagOfWordsA)

1670

In [182]:
doc_10k_str[2720768:2721042]

'cies are considered critical as disclosed in the critical accounting policies section of management’s discussion and analysis of financial condition and results of operations included in merck’s form 10-k because of the potential for a significant impact on the financial st'

In [50]:
i7 = "discussion and analysis of financial condition and results of operations"
#i7 = "financial condition and results of operations"
sec7_find = re.finditer(i7, doc_10k)

In [51]:
for x in sec7_find: 
    print(x.start())

30319
518530
716604


In [145]:
doc_10k[1087900:1132112]

'uding successfully integrating acquired businesses, could have an adverse effect on our business, financial condition and results of operations.&#160; Furthermore, we make strategic divestitures from time to time. In the case of divestitures, we may agree to indemnify acquiring parties for certain liabilities arising from our former businesses. These divestitures may also result in continued financial involvement in the divested businesses following the transaction, including through guarantees or other financial arrangements.&#160; Lower performance by those divested businesses could affect our future financial results.</span></div><div><span><br/></span></div><div><span style="color:#000000;font-family:\'Times New Roman\',sans-serif;font-size:10pt;font-weight:700;line-height:120%">Union disputes or other labor matters could adversely affect our operations and financial results.</span></div><div><span style="color:#000000;font-family:\'Times New Roman\',sans-serif;font-size:10pt;font

In [462]:
#i7 = 'management&#8217;s discussion and analysis of financial condition and results of operations'
i7 = "s discussion and analysis of financial condition and results of operations"
sec7_find = re.finditer(i7, doc_10k_str)
i7a = 'quantitative and qualitative disclosures about market risk' 
sec7a_find = re.finditer(i7a, doc_10k_str)
    
# moving keyword range to dataframe
sec7_start_range = []
sec7_end_range = []
for x in sec7_find: 
    #print(x.start())
    #print(x.end())
    sec7_start_range.append(x.start())
    sec7_end_range.append(x.end())
    sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
sec7a_start_range = []
sec7a_end_range = []    
for x in sec7a_find: 
    #print(x.start())
    #print(x.end())
    sec7a_start_range.append(x.start())
    sec7a_end_range.append(x.end())
    sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})
    
#qc check
for index, row in sec7_df.iterrows():
    print(row['start'], row['end'])
    print('item 7')
for index, row in sec7a_df.iterrows():
    print(row['start'], row['end'])
7 125654 125728
7 591991 592065
7 593414 593488
7a 127199 127257
7a 632141 632199

125654 125728
item 7
591991 592065
item 7
593414 593488
item 7
127199 127257
632141 632199


In [475]:
    
# get possible ranges 
possible_start_range = []
possible_end_range = []
for index, row7a in sec7a_df.iterrows():
    filtered = []
    for i, row7 in sec7_df.iterrows():
        if row7['end'] < row7a['start']:
            filtered.append(row7['end'])
            
    if not filtered:
        continue # nothing before item 7a
    #print(filtered)
    max_y = max(filtered)
    possible_start_range.append(max_y)
    possible_end_range.append(row7a['start']-12)
    
    # qc 
    #for x in possible_start_range:
    #    print(x)
    
    # get the largest possible range
possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
possible_df['delta'] = possible_df['end'] - possible_df['start']
    
possible_df = possible_df.loc[possible_df['delta'].idxmax()]


In [476]:
possible_df

start    593488
end      632129
delta     38641
Name: 1, dtype: int64

In [477]:
item_7_content = doc_10k_str[possible_df['start']:possible_df['end']]
sss = preprocess(item_7_content)
bagOfWordsA = sss.split(' ')

In [478]:
len(bagOfWordsA)

1802

In [472]:
bagOfWordsA[1700:1803]


['expense',
 'attribution',
 'three',
 'months',
 'six',
 'months',
 'ended',
 'june',
 'june',
 'subscription',
 'agreements',
 'subscription',
 'agreements',
 'involve',
 'physical',
 'settlement',
 'fixed',
 'number',
 'qualifies',
 'equity',
 'classification',
 'accounting',
 'standards',
 'codification',
 'asc',
 'derivatives',
 'hedging',
 'therefore',
 'periodically',
 'remeasured',
 'fair',
 'value',
 'backstop',
 'agreement',
 'backstop',
 'agreement',
 'involves',
 'conditional',
 'obligation',
 'company',
 'must',
 'settle',
 'issuing',
 'variable',
 'number',
 'shares',
 'monetary',
 'value',
 'predominantly',
 'based',
 'variations',
 'something',
 'fair',
 'value',
 'company',
 'shares',
 'initially',
 'subsequently',
 'measured',
 'fair',
 'value',
 'accounting',
 'standards',
 'codification',
 'asc',
 'distinguishing',
 'liabilities',
 'equity',
 'recent',
 'accounting',
 'pronouncements',
 'management',
 'believe',
 'recently',
 'issued',
 'yet',
 'effective',
 'accoun